In [ ]:
#Installs
!pip install xlrd
!pip install openpyxl
!pip install geneticalgorithm
!pip install pyeasyga

In [ ]:
#Imports
import numpy as np
import pandas as pd 
import math
#from datetime import datetime
#from contextlib import redirect_stdout #For the log files

import random

import matplotlib
import matplotlib.pyplot as plt
from matplotlib import rc
rc("text", usetex=False)


%matplotlib inline
import seaborn as sns

##Kmeans
from sklearn.cluster import KMeans
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler

##GMM
from sklearn.mixture import GaussianMixture

##DBSCAN
from sklearn.cluster import DBSCAN

##OPTICS
from sklearn.cluster import OPTICS
from sklearn.datasets import make_blobs

##Mean-Shift
from sklearn.cluster import MeanShift, estimate_bandwidth

##GA
from pyeasyga import pyeasyga
from scipy.stats import norm

##Likelihood 
from scipy.stats import norm
import scipy.stats as stats
import copy


In [ ]:
file_name = "../input/tfm-data/KBOXSales-DataTestCTES-25032021 2.xlsx"
df = pd.read_excel(file_name, engine='openpyxl', sheet_name='KMMVPRBQvm')
df.drop(['Unnamed: 92', 'Unnamed: 93',	'Unnamed: 94',	'Unnamed: 95',	'Unnamed: 96', 
         'Unnamed: 97',	'Unnamed: 98',	'Unnamed: 99'], axis = 1, inplace=True)

df_s = df[:5495].copy() 
df_s.head()

In [ ]:
train_x = df_s[['CodigoCLI', 'Vta19', 'VtaTar19', 'VtaCos19', 'Rappel19', 'MBto19', 
                '%ContVta19', 'CtVta19', '%ContMBto19', 'CtMBto19', ##kbox data
                'PosAbcVta19', '%PosAbcVta19', 'AcuVta19', '%AcuVta19', 'PosAbcMBto',
                '%PosAbcMBto19', 'AcuMBto19', '%AcuMBto19',
                
                'Key-P20', 'Key-B20', 'Key-V20', 'Key-QV20', 'Key-QM20', 'Key-R20', 'Key-WCI20',
                'KHR20', 'KVP20', 'KRB20', 'KeyQVM20', 'Short-KHR20', 'Short-KVP20', 'Short-KRB20', 'Short-KeyQVM20',]].copy()

train_x.dropna(axis=0, inplace=True)
train_x.reset_index(drop = True, inplace=True)

#X = train[['CodigoCLI', 'Vta19', 'VtaTar19', 'VtaCos19', 'Rappel19', 'MBto19']].copy()
X = train_x[['Vta19', 'VtaTar19', 'VtaCos19', 'Rappel19', 'MBto19', 
             '%ContVta19', 'CtVta19', '%ContMBto19', 'CtMBto19', ##kbox data
             'PosAbcVta19', '%PosAbcVta19', 'AcuVta19', '%AcuVta19', 'PosAbcMBto',
             '%PosAbcMBto19', 'AcuMBto19', '%AcuMBto19',]].copy()

scaler = MinMaxScaler(feature_range=(-1, 1))
X_scaled = scaler.fit_transform(X)


## 1) GMM 

In [ ]:
num = 1

for i in range(7):
    reg = 1e-5 #* 100 * (y+1)

    for y in range(5):
        # training gaussian mixture model 
        scaler = MinMaxScaler(feature_range=(-1, 1))
        X_scaled = scaler.fit_transform(X)
        #reg = 1e-5 * 100 * (y+1)
        gmm = GaussianMixture(n_components=i+1, reg_covar= reg)
        gmm.fit(X_scaled)
        
        #predictions from gmm
        labels = gmm.predict(X_scaled)
        frame = pd.DataFrame(X_scaled)
        frame['cluster'] = labels
        
        color=['blue','green','cyan', 'black']
        for k in range(0,4):
            X_scaled = frame[frame["cluster"]==k]
            plt.scatter(X_scaled.index.values, X_scaled.iloc[:,1],)
            #plt.scatter(X.iloc[:,2], X.iloc[:,3], c=color[k])
        # BOTTOM LABEL
        plt.figtext(0.5, 0, "Comps:{}, Reg_covar:{}".format(i+1, reg), ha="center", fontsize=7, bbox={"facecolor":"white", "alpha":0.5, "pad":5})
        plt.savefig('./gmm_film{}.png'.format(num))
        plt.clf()
        #plt.show()
        reg = reg * 10
        num = num + 1

## 2) DBSAN Grid Search

In [ ]:
num = 1
epsilon = 0.1

for i in range(7):
    min_s = 50

    for y in range(5):
        # training gaussian mixture model 
        scaler = MinMaxScaler(feature_range=(-1, 1))
        X_scaled = scaler.fit_transform(X)
        labels = DBSCAN(eps = epsilon, min_samples = min_s).fit_predict(X_scaled)
        
        #predictions from gmm
        #labels = gmm.predict(X_scaled)
        frame = pd.DataFrame(X_scaled)
        frame['cluster'] = labels
        
        color=['blue','green','cyan', 'black']
        for k in range(0,4):
            X_scaled = frame[frame["cluster"]==k]
            plt.scatter(X_scaled.index.values, X_scaled.iloc[:,1],)
            #plt.scatter(X.iloc[:,2], X.iloc[:,3], c=color[k])
        # BOTTOM LABEL
        plt.figtext(0.5, 0, "Epsilon:{}, Min_samples:{}".format(epsilon, min_s), ha="center", fontsize=7, bbox={"facecolor":"white", "alpha":0.5, "pad":5})
        plt.savefig('./dbscan_film{}.png'.format(num))
        #plt.show()

        #plt.clf()
        min_s = min_s + 30
        
        num = num + 1
        print(min_s)
    epsilon = epsilon + 0.2
    print(epsilon)


## 3) Genetic Methods

In [ ]:
X = train_x[['Vta19', 'VtaTar19', 'VtaCos19', 'Rappel19', 'MBto19', 
             '%ContVta19', 'CtVta19', '%ContMBto19', 'CtMBto19', ##kbox data
             'PosAbcVta19', '%PosAbcVta19', 'AcuVta19', '%AcuVta19', 'PosAbcMBto',
             '%PosAbcMBto19', 'AcuMBto19', '%AcuMBto19',]].copy()

scaler = MinMaxScaler(feature_range=(-1, 1))
X_scaled = scaler.fit_transform(X)

data = X_scaled
K = 4

In [ ]:
# define and set the GA's crossover operation
def crossover_function(parent_1, parent_2):
   
    child_1a = random.sample(parent_1, 2) #6:3
    child_1b = [i for i in parent_2 if i not in child_1a]
    child_1b = random.sample(child_1b, 2) #3:1, 4:2, 5:3, 6:3
    child_1 = child_1a + child_1b
    
    child_2a = random.sample(parent_2, 2) #6:3
    child_2b = [i for i in parent_1 if i not in child_2a]
    child_2b = random.sample(child_2b, 2) #3:1, 4:2, 5:3, 6:3
    child_2 = child_2a + child_2b       

    return child_1, child_2


# define and set the GA's mutation operation
def mutate_function(individual):
    mutate_index1 = random.randrange(len(individual))
    mutate_index2 = random.randrange(len(individual))
    #print(mutate_index1, mutate_index2)
    individual[mutate_index1], individual[mutate_index2] = individual[mutate_index2], individual[mutate_index1]
    #return individual[mutate_index1], individual[mutate_index2]

In [ ]:
def master_mu_sigma_population_creator(data):
    
    master_mu_list_1 = []
    master_mu_list_2 = []
    
    master_sigma_list_1 = []
    master_sigma_list_2 = []
    
    for i in range(17):
    
        mu_list_1 = []
        mu_list_2 = []

        sigma_list_1 = []
        sigma_list_2 = []

        mu_high = np.mean(data[:,i]) + np.std(data[:,i])
        mu_low = np.mean(data[:,i]) - np.std(data[:,i])

        sigma_high = np.var(data[:,i]) + np.std(data[:,i])  
        sigma_low =  np.var(data[:,i]) - np.std(data[:,i]) 

        for i in range(25):

            mu_1 = np.random.uniform(low = mu_low, high = mu_high, size=K) # mean
            sigma_1 = np.random.uniform(low = sigma_low, high = sigma_high, size=K) # standard deviaiton
            #mu_1 = np.random.uniform(low=42.0, high=95.0, size=K) # mean
            #sigma_1 = np.random.uniform(low=5.0, high=10.0, size=K) # standard deviaiton
            mu_list_1.append(list(mu_1))
            sigma_list_1.append(list(sigma_1))
        
        master_mu_list_1.append(mu_list_1)
        master_sigma_list_1.append(sigma_list_1)

        for y in range(25):
            mu_2 = np.random.uniform(low = mu_low, high = mu_high, size=K) # mean
            sigma_2 = np.random.uniform(low = sigma_low, high = sigma_high, size=K) # standard deviaiton
            #mu_2 = np.random.uniform(low=42.0, high=95.0, size=K) # mean
            #sigma_2 = np.random.uniform(low=5.0, high=10.0, size=K) # standard deviaiton
            mu_list_2.append(list(mu_2))
            sigma_list_2.append(list(sigma_2))
        
        master_mu_list_2.append(mu_list_2)
        master_sigma_list_2.append(sigma_list_2)


        
    return (master_mu_list_1, master_mu_list_2, master_sigma_list_1, master_sigma_list_2)

In [ ]:
population = master_mu_sigma_population_creator(data)
parent_1, parent_2 = population[0], population[1]
parent_3, parent_4 = population[2], population[3]

In [ ]:
def master_crossover_mutation(master_parent_1, master_parent_2, crossover_prob = 0.8, mutation_prob = 0.2):
    
    new_populations = []

    for i in range(17):
        
        new_population = []
        population_size = 25
        parent_1, parent_2 = master_parent_1[i], master_parent_2[i]
        child_1, child_2 = parent_1, parent_2

        for i in range(50):
            while len(new_population) < population_size:

                can_crossover = random.random() <= crossover_prob
                can_mutate = random.random() <= mutation_prob

                if can_crossover:
                    child_1[i], child_2[i] = crossover_function(parent_1[i], parent_2[i])

                if can_mutate:
                    mutate_function(child_1[i])
                    mutate_function(child_2[i])

                new_population.append(child_1[i])
                if len(new_population) < population_size:
                    new_population.append(child_2[i])
        
        new_populations.append(new_population)

    return new_populations

In [ ]:
new_population_mu = master_crossover_mutation(master_parent_1 = parent_1, master_parent_2 = parent_2)
new_population_sigma = master_crossover_mutation(master_parent_1 = parent_3, master_parent_2 = parent_4)

In [ ]:
def plot_distributions(data, data_sampled, mu, sigma, K, color="green", color_sampled="red", name='plot.png'): 
    data_sampled = np.clip(data_sampled, np.min(data), np.max(data))
    plt.hist(data, bins=15, color=color, alpha=0.45, density=True)
    plt.hist(data_sampled, bins=15, range=(np.min(data), np.max(data)), color=color_sampled, alpha=0.45, density=True)
    for k in range(K):
        curve = np.linspace(mu[k] - 10*sigma[k], mu[k] + 10*sigma[k], 100)
        color = np.random.rand(3)
        plt.plot(curve, stats.norm.pdf(curve, mu[k], sigma[k]), color=color, linestyle="--", linewidth=3)
    plt.ylabel(r"$p(x)$")
    plt.xlabel(r"$x$")
    plt.tight_layout()
    plt.xlim(20, 120)
    plt.savefig(name, dpi=200)
    plt.show()

def plot_likelihood(nll_list, num):
    plt.plot(np.arange(len(nll_list)), nll_list, color="black", linestyle="--", linewidth=3)
    plt.ylabel(r"(negative) log-likelihood")
    plt.xlabel(r"iteration")
    plt.tight_layout()
    plt.xlim(0, len(nll_list))
    #plt.savefig('nll1.png', dpi=200)
    plt.savefig('./real_data_{}.png'.format(num), dpi = 200)
    plt.show()

In [ ]:
def master_likelihood_calculator(master_dt, master_mu_list, master_sigma_list):    
    #data = np.genfromtxt(dt, delimiter=',', skip_header=1)#[:,-2]
    #data = data[:,-
    master_final_mu = []
    master_final_sigma = []
    master_final_pi = []
    
    final_mu_list = []
    final_sigma_list = []
    final_pi_list = []
    
    for i in range(17):
        data = master_dt[:,i]
        N = data.shape[0]
        K = 4 # two components GMM
        tot_iterations = 100 # stopping criteria

        # Step-1 (Init)
        last_nll_list = list() ## Used to collect the last nulls for the mu
        complete_nll_list = list() ## Used to collect all the null lists


        num = 1

        mu_list = master_mu_list[i]
        sigma_list = master_sigma_list[i]

        for mu_elem, sigma_elem in zip(mu_list, sigma_list):
           
            pi = np.ones(K) * (1.0/K) # mixing coefficients
            r = np.zeros([K,N]) # responsibilities
            nll_list = list() # store the neg log-likelihood

            mu = mu_elem
            sigma = sigma_elem
            

            for iteration in range(tot_iterations):  
                # Step-2 (E-Step)
                for k in range(K):   
                    r[k,:] = pi[k] * norm.pdf(x=data, loc=mu[k], scale=sigma[k])
                r = r / np.sum(r, axis=0) #[K,N] -> [N]

                # Step-3 (M-Step)
                N_k = np.sum(r, axis=1) #[K,N] -> [K]
                for k in range(K): 
                    # update means
                    mu[k] = np.sum(r[k,:] * data) / N_k[k]        
                    # update variances
                    numerator = r[k] * (data - mu[k])**2
                    sigma[k] = np.sqrt(np.sum(numerator) / N_k[k])        
                # update weights
                pi = N_k/N 

                likelihood = 0.0

                for k in range(K):
                    likelihood += pi[k] * norm.pdf(x=data, loc=mu[k], scale=sigma[k])

                nll_list.append(-np.sum(np.log(likelihood)))        


                # Check for invalid negative log-likelihood (NLL)
                # The NLL is invalid if NLL_t-1 < NLL_t
                # Note that this can happen for round-off errors.
                if(len(nll_list)>=2):
                    if(nll_list[-2]<nll_list[-1]): raise Exception("[ERROR] invalid NLL: "+str(nll_list[-2:]))

                #print("Iteration: " + str(iteration) + "; NLL: " + str(nll_list[-1]))
                #print("Mean " + str(mu) + "\nStd " + str(sigma) + "\nWeights " + str(pi) + "\n")     

                # Step-4 (Check)
                if(iteration == tot_iterations-1): 
                    last_nll_list.append(nll_list[-1])
                    break # check iteration
                    
            if min(mu) > 0: 
                final_mu_list.append(mu)
            if min(sigma) > 0: 
                final_sigma_list.append(sigma)
            if min(pi) > 0: 
                final_pi_list.append(pi)
        
            #plot_likelihood(nll_list, num)
            num = num +1
        #data_gmm = sampler(pi, mu, sigma, N=1000)
        #plot_distributions(data, data_gmm, mu, sigma, K, color="green", color_sampled="red", name="plot_sampler.png")
        master_final_mu.append(final_mu_list)
        master_final_sigma.append(final_sigma_list)
        master_final_pi.append(final_pi_list)
        
        
    return (master_final_mu, master_final_sigma, master_final_pi)

In [ ]:
new_population_mu_copy = copy.deepcopy(new_population_mu)
new_population_sigma_copy = copy.deepcopy(new_population_sigma)

params = master_likelihood_calculator(data, new_population_mu_copy, new_population_sigma_copy)

In [ ]:
params_1 = [ [[x for x in y if not np.isnan(x)] for y in z] for z in params[0] ]
params_2 = [ [[x for x in y if not np.isnan(x)] for y in z] for z in params[1] ]
params_3 = [ [[x for x in y if not np.isnan(x)] for y in z] for z in params[2] ]


In [ ]:
for i in range(17):
    params_1[i] = (list(filter(lambda x: x, params_1[i])))
    params_2[i] = (list(filter(lambda x: x, params_2[i])))
    params_3[i] = (list(filter(lambda x: x, params_3[i])))

In [ ]:
mu_means = []
sigma_means = []
pi_means = []

for i in range(17):
    mu_means.append(np.average(params_1[i], axis = 0))
    sigma_means.append(np.mean(params_2[i], axis = 0))
    pi_means.append(np.average(params_3[i], axis = 0))


In [ ]:
# training gaussian mixture model 
gmm = GaussianMixture(n_components=4, reg_covar=1e-4, 
                      means_init = np.transpose(mu_means), 
                      weights_init = np.mean(pi_means, axis=0))
gmm.fit(X_scaled)

In [ ]:
no_init = []
init = []

for i in range(15):
    y = train_x.iloc[:, 18+i].copy()
    labelEncoder = LabelEncoder()
    labelEncoder.fit(y)
    y = labelEncoder.transform(y)
    y = np.array(y)
    
    correct = 0
    for i in range(len(X_scaled)):
        predict_me = np.array(X_scaled[i])
        predict_me = predict_me.reshape(-1, len(predict_me))
        prediction = gmm.predict(predict_me)
        if prediction[0] == y[i]:
            correct += 1
            
    init.append(correct/len(X_scaled))
    print(correct/len(X_scaled))